### Разработка нейросетевых функций. Операция Convolution Transpose

### Дополнительное задание

#### Описание параметров

in_channels (int) – Number of channels in the input image\
out_channels (int) – Number of channels produced by the convolution\
kernel_size (int or tuple) – Size of the convolving kernel\
stride (int or tuple, optional) – Stride of the convolution. Default: 1\
padding (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0\
padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'\
dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1\
groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1\
bias (bool, optional) – If True, adds a learnable bias to the output. Default: True\

In [8]:
import torch
import numpy as np
import warnings
warnings.simplefilter("ignore")

def myTranspConv2d(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, 
                   padding_mode='zeros'):
    def Conv(matrix):
        stride = 1

        pad = kernel_size - 1
        result_matrix = []
        for matr in matrix:
            zero_tensor = np.zeros((((matr.shape[0]-1)*(transp_stride)+1), ((matr.shape[1]-1)*(transp_stride)+1)))
            for a in range (0, zero_tensor.shape[0], transp_stride):
                for b in range (0, zero_tensor.shape[1], transp_stride):
                    zero_tensor[a][b] = matr[a//(transp_stride)][b//(transp_stride)]

            pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
            result_matrix.append(pad_matr)
        matrix = torch.tensor(result_matrix)

        if bias == True:
            bias_val = torch.rand(out_channels)
        else:
            bias_val = torch.zeros(out_channels)

        if (padding_mode == 'zeros'):
            pad = torch.nn.ZeroPad2d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'reflect'):
            pad = torch.nn.ReflectionPad2d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'replicate'):
            pad = torch.nn.ReplicationPad2d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'circular'):
            pad = torch.nn.CircularPad2d(padding)
            matrix = pad(matrix)

        filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

        filter_for_transpose = []
        for j in range(out_channels):
            filter_in = []
            for i in range(in_channels):
                filter_in.append(np.flip(np.array(filter[j][i])))
            filter_for_transpose.append(filter_in)

        filter_for_transpose = torch.tensor(filter_for_transpose)
        filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)



        li = []
        for l in range(out_channels):
            feature_map = np.array([]) 
            for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): 
                for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
                    summa = 0
                    for c in range (in_channels):
                        val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*
                                        dilation+1:dilation]
                        mini_sum = (val*filter[l][c]).sum()
                        summa = summa + mini_sum
                    feature_map = np.append(feature_map, float(summa + bias_val[l])) 
            li.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1,
                                          (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))


        return np.array(li), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_val))

    return Conv

In [4]:
img1 = torch.rand(3, 5, 6)
img2 = torch.rand(1, 28, 28)
img3 = torch.rand(7, 10, 10)

In [9]:
Func = myTranspConv2d(in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, bias=True,)
result, kernel, bias_val = Func(img1)
torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True,)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(img1).data), 2))
torchResult == myResult

True

In [10]:
Func2 = myTranspConv2d(in_channels=1, out_channels=2, kernel_size=4, transp_stride=3, bias=True)
result2, kernel2, bias_val2 = Func2(img2)
torchFunction2 = torch.nn.ConvTranspose2d(in_channels=1, out_channels=2, kernel_size=4, stride=3, bias=True)
torchFunction2.weight.data = kernel2
torchFunction2.bias.data = bias_val2
myResult2 = str(np.round(result2, 2))
torchResult2 = str(np.round(np.array(torchFunction2(img2).data), 2))
torchResult2 == myResult2

True

In [11]:
Func3 = myTranspConv2d(in_channels=7, out_channels=1, kernel_size=3, transp_stride=5, bias=True)
result3, kernel3, bias_val3 = Func3(img3)
torchFunction3 = torch.nn.ConvTranspose2d(in_channels=7, out_channels=1, kernel_size=3, stride=5, bias=True)
torchFunction3.weight.data = kernel3
torchFunction3.bias.data = bias_val3
myResult3 = str(np.round(result3, 2))
torchResult3 = str(np.round(np.array(torchFunction3(img3).data), 2))
torchResult3 == myResult3

True